%

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import seaborn as sns

In [ ]:
from geopy.distance import geodesic
import numpy as np
# %%

In [ ]:
df = pd.read_csv('GPS_ruta.csv')
df_new = df.copy()
# %%
#cómo están los datos?
df_new
# Calcula el recuento de registros por ruta
conteo_rutas = df_new['ruta'].value_counts()

Crea el gráfico de barras

In [ ]:
plt.figure(figsize=(12, 6))
conteo_rutas.plot(kind='bar', color='skyblue')
plt.xlabel('Ruta')
plt.ylabel('Número de Registros')
plt.title('Número de Registros por Ruta')
plt.xticks(rotation=90, fontsize = 6)  # Rotar las etiquetas del eje X para mayor legibilidad
plt.show()
#%%
df_new = df_new.drop_duplicates(subset=['id_unidad', 'latitud', 'longitud'], keep=False)
# Calcula el recuento de registros por ruta
conteo_rutas = df_new['ruta'].value_counts()

Crea el gráfico de barras

In [ ]:
plt.figure(figsize=(12, 6))
conteo_rutas.plot(kind='bar', color='skyblue')
plt.xlabel('Ruta')
plt.ylabel('Número de Registros filtradas')
plt.title('Número de Registros por Ruta')
plt.xticks(rotation=90, fontsize = 6)  # Rotar las etiquetas del eje X para mayor legibilidad
plt.show()
#%%
# Calcula la proporción de información por ruta
proporcion_rutas = df_new['ruta'].value_counts(normalize=True)

Crea el gráfico de tarta

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(proporcion_rutas, labels=proporcion_rutas.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.axis('equal')  # Para que el gráfico de tarta sea circular
plt.title('Proporción de Información por Ruta')
plt.show()
tamaño = df.shape
tamaño
#%%
tipos_de_datos = df.dtypes
tipos_de_datos
df
#%%
cont_zeros = df['contadorpasajeros'].value_counts().get(0,0)
cont_zeros
#%%
#Veamos si hay irregularidades,
cuenta_de_filas = df['velocidad'].count()
cuenta_de_filas
#%%
cont_zeros/cuenta_de_filas
# %%
#quiero checar cuantos ceros hay en la columna velocidad
cuenta_de_ceros = df['velocidad'].value_counts().get(0, 0)
cuenta_de_ceros
# %%
cuenta_de_ceros = (df['velocidad'] == 0).sum()
cuenta_de_ceros
# %%
cuenta_de_ceros/cuenta_de_filas
#Como el 61% contienen ceros, nos daremos la tarea de rehacer esta columna
#Vamos a calcular la velocidad a mano usando
#la formula de la distancia euclediana
# %%
#Vamos a agrupar por ruta 
# y vamos a agarrar las herramientas 
# necesarias para la distancia
columnas_deseadas = ['timestamp', 'ruta', 'latitud', 'longitud']
grupos = df.groupby('ruta')[columnas_deseadas]
#%%
#si jalo?
for ruta, grupo in grupos:
    print(f'Ruta: {ruta}')
    print(grupo)
    print('\n') 
#%% Hay que calcular la distancia
#ESTO ESTA PENDIENTE!
# %%
# Ahora veamos cual es el intervalo
# tiempo por el que están separadas las rutas
# realmente son 30 sec?
# Asegúrate de que la columna 'timestamp' esté en formato datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

Ordena el DataFrame por 'ruta' y 'timestamp' para asegurarnos de que los registros estén en orden temporal dentro de cada ruta

In [ ]:
df.sort_values(by=['ruta', 'timestamp'], inplace=True)

Calcula la diferencia de tiempo entre registros dentro de cada ruta

In [ ]:
df['diferencia_tiempo'] = df.groupby('ruta')['timestamp'].diff()

Calcula el tiempo promedio entre registros para cada ruta

In [ ]:
tiempo_promedio_por_ruta = df.groupby('ruta')['diferencia_tiempo'].mean()
tiempo_promedio_por_ruta = tiempo_promedio_por_ruta.sort_values(ascending=False)
#%%
# Muestra el resultado

In [ ]:
tiempo_promedio_por_ruta = df.groupby('ruta')['diferencia_tiempo'].mean()
tiempo_promedio_por_ruta = tiempo_promedio_por_ruta.dt.total_seconds() / 60  # Convierte a minutos

Ordena los datos de forma descendente

In [ ]:
tiempo_promedio_por_ruta = tiempo_promedio_por_ruta.sort_values(ascending=False)

Crea un gráfico de barras para mostrar el tiempo promedio por ruta

In [ ]:
plt.figure(figsize=(12, 6))
tiempo_promedio_por_ruta.plot(kind='bar', color='skyblue')
plt.title('Tiempo Promedio por Ruta')
plt.xlabel('Ruta')
plt.ylabel('Tiempo Promedio (minutos)')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plt.show()

ONCLUIMOS que NO todas estan separadas por 30 segundos hay mucha variacion en los datos<br>
%%<br>
 que tan estadisticamente confiables son estas diferencias, en otras palabras <br>
ue tan grande es la N

In [ ]:
df['ocurrencias_ruta'] = df.groupby('ruta')['ruta'].transform('count')
df
# %%
conteo_rutas = df['ruta'].value_counts().reset_index()
conteo_rutas.columns = ['ruta', 'Número de Ocurrencias']
conteo_rutas
#La ruta 1 sí prende su gps; la ruta 34 no. o por que razón la ruta 34 aparece tan poco,
#veamos cuantos vehiculos tienen
# %%
#Ahora hay que unir nuestros dos df y lo hacemos con la ruta

In [ ]:
nuevo_df = pd.merge(conteo_rutas, tiempo_promedio_por_ruta, on='ruta', how='left')
nuevo_df = nuevo_df.rename(columns={'Número de Ocurrencias': 'Número de Ocurrencias', 'ruta': 'ruta'})
nuevo_df
##RESULTADO IMPORTANTE
#%%
df_sorted = nuevo_df.sort_values(by='Número de Ocurrencias', ascending=True)

Define cuántas de las rutas con menos ocurrencias quieres excluir (en este caso, las 10 más bajas)

In [ ]:
rutas_a_excluir = 30

Filtra el DataFrame para excluir las rutas con menos ocurrencias

In [ ]:
df_filtrado = df_sorted.iloc[rutas_a_excluir:].sort_index(ascending=True)
df_filtrado['ruta']
#%%
tiempo_promedio_data = nuevo_df['diferencia_tiempo'].dt.total_seconds() / 60  # Convierte a minutos
tiempo_promedio_data
#%%
df_filtrado['diferencia_tiempo'] = pd.to_datetime(df['diferencia_tiempo'])
# Divide los datos en dos DataFrames separados para facilitar el trazado de los ejes Y
ruta_data = df_filtrado['ruta']
ocurrencias_data = df_filtrado['Número de Ocurrencias']
tiempo_promedio_data = df_filtrado['diferencia_tiempo'].dt.total_seconds() / 60  # Convierte a minutos
valor_promedio_tiempo = tiempo_promedio_data.mean()

Crea una figura con dos ejes Y

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

Configura el primer eje Y (número de ocurrencias)

In [ ]:
ax1.bar(ruta_data, ocurrencias_data, color='skyblue', alpha=0.7, label='Número de Ocurrencias')
ax1.set_xlabel('Ruta')
ax1.set_ylabel('Número de Ocurrencias', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticklabels(ruta_data, rotation=45)
ax1.legend(loc='upper left')

Configura el segundo eje Y (tiempo promedio)

In [ ]:
ax2 = ax1.twinx()
ax2.plot(ruta_data, tiempo_promedio_data, color='orange', marker='o', label='Tiempo Promedio (segundos)')
ax2.set_ylabel('Tiempo Promedio (segundos)', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')
ax2.legend(loc='upper right')
ax1.set_xticklabels(ruta_data, rotation=45, fontsize=6)  # Ajusta el tamaño de fuente aquí
#Creamos una line 
ax1.annotate(f'Promedio: {valor_promedio_tiempo:.2f} segundos', xy=(1, 0), xycoords=('axes fraction', 'axes fraction'),
             fontsize=14, color='orange', horizontalalignment='right')

In [ ]:
ax1.legend(loc='upper left')

Título y etiquetas de los ejes

In [ ]:
plt.title('Número de Ocurrencias y Tiempo Promedio por Ruta')
plt.tight_layout()

In [ ]:
plt.show()
# %%
conteo_tipos_vehiculos = df.groupby('ruta')['tipounidad'].nunique().reset_index()
conteo_tipos_vehiculos.columns = ['ruta', 'Número de Tipos de Vehículos Diferentes']
conteo_tipos_vehiculos = conteo_tipos_vehiculos.sort_values(by="Número de Tipos de Vehículos Diferentes",ascending=False)
conteo_tipos_vehiculos
# %%
vehiculos_ruta_1 = conteo_tipos_vehiculos[conteo_tipos_vehiculos['ruta'] == 'RUTA 34']['Número de Tipos de Vehículos Diferentes'].values[0]
vehiculos_ruta_1
# %%
new_df = pd.merge(nuevo_df, conteo_tipos_vehiculos, on='ruta', how='left')
new_df = new_df.rename(columns={'Número de Ocurrencias': 'Número de Ocurrencias', 'ruta': 'ruta'})
new_df
# %%
# 1. Ordena el DataFrame por id_unidad y timestamp
df.sort_values(by=['id_unidad', 'timestamp'], inplace=True)
df
df_segundo_filtrado = df.copy
#%%
conteo_id_u = df['id_unidad'].value_counts()
conteo_id_u
#%%
plt.figure(figsize=(10,7))
plt.violinplot(conteo_id_u.values)
plt.title('Boxplot de Número de Ocurrencias por Id_Unidad')
plt.ylabel('Número de Ocurrencias')
plt.show()
#Preguntas pendientes
# Más importante para hacerla a semovi, el que tengan muchas ocurrencias por vehiculo implica que son los que mas prenden su gps?
#  cuales son los vehiculos que mas estan registrados
# pendiente: 
# calcular la distancia euclediana
#   preguntas: 
#       cual es el vehiculo que se pasa más veces de la velocidad permitida
#       cuales son los 
# objetivo hipotetico:
# tratar de relacionar la distancia obtenida

MPIEZA EL OTRO ANALISISSSSSS

In [ ]:
""
#%%
#%%
# Elimina las filas duplicadas
df_segundo_filtrado = df.drop_duplicates(subset=['id_unidad', 'latitud', 'longitud'], keep=False)
#%%
# Calcula el número de ocurrencias por ruta
ocurrencias_por_ruta = df['ruta'].value_counts()
ocurrencias_por_ruta
#%%
# Ordena el DataFrame en función del número de ocurrencias en orden descendente
df_segundo_filtrado = df.sort_values(by='ocurrencias_ruta', ascending=False)
df_segundo_filtrado
#%%
# Selecciona las rutas que deseas mantener (las que no están en las 30 rutas con menos ocurrencias)
rutas_a_mantener = ocurrencias_por_ruta.index[:30]
rutas_a_mantener
#%%
# Filtra el DataFrame original para obtener las rutas seleccionadas
df_segundo_filtrado = df_segundo_filtrado[df_segundo_filtrado['ruta'].isin(rutas_a_mantener)]
df_segundo_filtrado

%

In [ ]:
df = df[df['ruta'] == 'RUTA 1']

In [ ]:
filas_duplicadas = df[df.duplicated(subset=['id_unidad', 'latitud', 'longitud'], keep=False)]

Elimina las filas duplicadas

In [ ]:
df = df.drop_duplicates(subset=['id_unidad', 'latitud', 'longitud'], keep=False)

In [ ]:
df

%%

In [ ]:
def calculate_distance(group):
    latitudes = group['latitud'].tolist()
    longitudes = group['longitud'].tolist()
    distances = [0]  # Inicializamos la lista de distancias con un valor cero para la primera fila
    for i in range(1, len(latitudes)):
        coords_1 = (latitudes[i - 1], longitudes[i - 1])
        coords_2 = (latitudes[i], longitudes[i])
        distance = geodesic(coords_1, coords_2).kilometers
        distances.append(distance)
    return pd.Series(distances, index=group.index)

In [ ]:
df['distancia_km'] = df.groupby('id_unidad').apply(calculate_distance).reset_index(level=0, drop=True)

%<br>
%%<br>
3. Calcula el tiempo transcurrido en minutos

In [ ]:
df_segundo_filtrado['timestamp'] = pd.to_datetime(df_segundo_filtrado['timestamp'])
df_segundo_filtrado['tiempo_minutos'] = df_segundo_filtrado.groupby('id_unidad')['timestamp'].diff().dt.total_seconds() / 60
#%%
# 4. Calcula la velocidad en km/h
df_segundo_filtrado['velocidad'] = df_segundo_filtrado['distancia_km'] / df_segundo_filtrado['tiempo_minutos']
#%%
# Puedes rellenar los valores NaN en la primera fila de cada id_unidad con 0 si es necesario
df_segundo_filtrado['velocidad'] = df_segundo_filtrado.groupby('id_unidad')['velocidad'].transform(lambda x: x.fillna(0))

% Elimina las columnas auxiliares si no las necesitas

In [ ]:
df_segundo_filtrado.drop(['distancia_km', 'tiempo_minutos'], axis=1, inplace=True)

Ahora df contiene la columna de velocidad reconstruida con distancia euclidiana y tiempo en minutos

%%

In [ ]:
df_segundo_filtrado['id_unidad'].value_counts()

%%

In [ ]:
df_segundo_filtrado.groupby(['id_unidad'])
# %%
df_speed_mean = df_segundo_filtrado['velocidad'].mean()
df_speed_mean

%

In [ ]:
df_segundo_filtrado['velocidad'].replace([np.inf, -np.inf], np.nan, inplace=True)

Cuenta los valores únicos y obtén el recuento de "inf" (valores infinitos)

In [ ]:
recuento_inf = df_segundo_filtrado['velocidad'].value_counts().get(np.nan, 0)
recuento_inf
#%%
recuento_inf = df_segundo_filtrado['velocidad'].apply(lambda x: x == float('inf')).sum()
recuento_inf
tot_velocidad = df_segundo_filtrado['velocidad'].count()
perc = recuento_inf / tot_velocidad
perc
# %%
df_speed_mean = df_segundo_filtrado['velocidad'].mean()
df_speed_mean
# %%
df
# %%
#Vamos a tomar los automoviles que pasan de 48km/h como vehiculos problematicos
ruta_1_df = df_segundo_filtrado['ruta']
ruta_1_df = df_segundo_filtrado[df_segundo_filtrado['velocidad']>48]

Esto nos sirve para visualizar los vehiculos anteriormente mencionados<br>
Crear un gráfico de dispersión

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=ruta_1_df, x='timestamp', y='velocidad', palette='viridis', s=10)
plt.title('Distribución de Velocidad por Id_Unidad a lo largo del Tiempo de la ruta 1')
plt.xlabel('Timestamp')
plt.ylabel('Velocidad (km/h)')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor legibilidad
for i, row in ruta_1_df.iterrows():
    plt.annotate(row['id_unidad'], (row['timestamp'], row['velocidad']), fontsize=8, ha='center', va='bottom')
plt.show()
#%%
#Vamos a ver las caracteristicas de estos vehiculos
df_new
# %%
# Convierte la columna de timestamp a tipo datetime si aún no está en ese formato
df['timestamp'] = pd.to_datetime(df['timestamp'])

Filtra el DataFrame para obtener solo las filas donde la velocidad supera el límite

In [ ]:
vehiculos_sobrepasan_limite = df[df['velocidad'] > 48]

Obtén la hora de cada registro

In [ ]:
vehiculos_sobrepasan_limite['hora'] = vehiculos_sobrepasan_limite['timestamp'].dt.hour

Cuenta cuántas veces se cometen actos de imprudencia a cada hora

In [ ]:
conteo_por_hora = vehiculos_sobrepasan_limite['hora'].value_counts()

Encuentra la hora con el mayor número de actos de imprudencia

In [ ]:
hora_mas_comun = conteo_por_hora.idxmax()
cantidad_actos_imprudencia = conteo_por_hora.max()

In [ ]:
print("La hora en la que se cometen más actos de imprudencia es aproximadamente a las", hora_mas_comun, "horas.")
print("En esa hora se cometen", cantidad_actos_imprudencia, "actos de imprudencia.")
# %%
# Cuál es la ruta que más veces sobrepasa el límite de velocidad
#
ruta_1_df['id_unidad'].value_counts()
#%%
# RESULTADOS GENERALES
# Cuál es el tipo de vehiculo que aparece más en estos vehiculos problematicos
# A que hora es en la que se cometen estos actos de imprudencia
# Filtra el DataFrame para obtener solo las filas donde la velocidad supera el límite
velocidad_limite = 48  # Supongamos que el límite de velocidad es de 48 km/h
vehiculos_sobrepasan_limite = df_new[df_new['velocidad'] > velocidad_limite]

Luego, cuentas cuántas veces aparece cada ruta en este subconjunto

In [ ]:
ruta_mas_sobrepasada = vehiculos_sobrepasan_limite['ruta'].value_counts().idxmax()
num_veces = vehiculos_sobrepasan_limite['ruta'].value_counts().max()
print("La ruta que más veces sobrepasa el límite de velocidad en df_new es:", ruta_mas_sobrepasada, "y lo sobrepasa", num_veces)

%%

In [ ]:
vehiculos_sobrepasan_limite = df_new[df_new['velocidad'] > velocidad_limite]

Luego, cuentas cuántas veces aparece cada ruta en este subconjunto

In [ ]:
top_10_rutas = vehiculos_sobrepasan_limite['ruta'].value_counts().nlargest(10)

Crea un gráfico de barras para mostrar las 10 rutas con más ocurrencias

In [ ]:
plt.figure(figsize=(12, 6))
top_10_rutas.plot(kind='bar', color='skyblue')
plt.title('Top 10 Rutas con Mayor Exceso del Límite de Velocidad')
plt.xlabel('Ruta')
plt.ylabel('Número de Ocurrencias')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plt.show()
#
##
#
#
#
#
#
#
#
#
#
#Analisis de la ruta 1
#
# %%
df
#%%
# 3. Calcula el tiempo transcurrido en minutos
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['tiempo_minutos'] = df.groupby('id_unidad')['timestamp'].diff().dt.total_seconds() / 60
#%%
# 4. Calcula la velocidad en km/h
df['velocidad'] = df['distancia_km'] / df['tiempo_minutos']
#%%
# Puedes rellenar los valores NaN en la primera fila de cada id_unidad con 0 si es necesario
df['velocidad'] = df.groupby('id_unidad')['velocidad'].transform(lambda x: x.fillna(0))

% Elimina las columnas auxiliares si no las necesitas

In [ ]:
df.drop(['distancia_km', 'tiempo_minutos'], axis=1, inplace=True)

Ahora df contiene la columna de velocidad reconstruida con distancia euclidiana y tiempo en minutos

%%

In [ ]:
df['id_unidad'].value_counts()

%%

In [ ]:
df.groupby(['id_unidad'])
# %%
df_speed_mean = df['velocidad'].mean()
df_speed_mean

%

In [ ]:
df['velocidad'].replace([np.inf, -np.inf], np.nan, inplace=True)

Cuenta los valores únicos y obtén el recuento de "inf" (valores infinitos)

In [ ]:
recuento_inf = df['velocidad'].value_counts().get(np.nan, 0)
recuento_inf
#%%
recuento_inf = df['velocidad'].apply(lambda x: x == float('inf')).sum()
recuento_inf
tot_velocidad = df['velocidad'].count()
perc = recuento_inf / tot_velocidad
perc
# %%
df_speed_mean = df['velocidad'].mean()
df_speed_mean
# %%
#Vamos a tomar los automoviles que pasan de 48km/h como vehiculos problematicos
ruta_1_df = df[df['ruta'] == 'RUTA 1']
ruta_1_df = df[df['velocidad']>48]
colores_unidades = ['blue' if unidad != '0011EAA' else 'green' for unidad in ruta_1_df['id_unidad']]

Esto nos sirve para visualizar los vehiculos anteriormente mencionados<br>
Crear un gráfico de dispersión

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(data=ruta_1_df, x='timestamp', y='velocidad', hue = colores_unidades, palette='viridis', s=10)
plt.title('Distribución de Velocidad por Id_Unidad a lo largo del Tiempo de la ruta 1')
plt.xlabel('Timestamp')
plt.ylabel('Velocidad (km/h)')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para mejor legibilidad
for i, row in ruta_1_df.iterrows():
    plt.annotate(row['id_unidad'], (row['timestamp'], row['velocidad']), fontsize=8, ha='center', va='bottom')
plt.show()
#%%
#Vamos a ver las caracteristicas de estos vehiculos
unidad_0011EAA = ruta_1_df[ruta_1_df['id_unidad'] == '0011EAA']
unidad_0011EAA
# %%
# Convierte la columna de timestamp a tipo datetime si aún no está en ese formato
df['timestamp'] = pd.to_datetime(df['timestamp'])

Filtra el DataFrame para obtener solo las filas donde la velocidad supera el límite

In [ ]:
vehiculos_sobrepasan_limite = df[df['velocidad'] > 48]

Obtén la hora de cada registro

In [ ]:
vehiculos_sobrepasan_limite['hora'] = vehiculos_sobrepasan_limite['timestamp'].dt.hour

Cuenta cuántas veces se cometen actos de imprudencia a cada hora

In [ ]:
conteo_por_hora = vehiculos_sobrepasan_limite['hora'].value_counts()

Encuentra la hora con el mayor número de actos de imprudencia

In [ ]:
hora_mas_comun = conteo_por_hora.idxmax()
cantidad_actos_imprudencia = conteo_por_hora.max()

In [ ]:
print("La hora en la que se cometen más actos de  Excesos de Velocidad es aproximadamente a las", hora_mas_comun, "horas.")
print("En esa hora se cometen", cantidad_actos_imprudencia, "actos de  Excesos de Velocidad.")

%<br>
Convierte la columna 'timestamp' a datetime

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

Filtra el DataFrame para obtener solo las filas donde la velocidad supera el límite

In [ ]:
vehiculos_sobrepasan_limite = df[df['velocidad'] > 48]

Obtén la hora de cada registro

In [ ]:
vehiculos_sobrepasan_limite['hora'] = vehiculos_sobrepasan_limite['timestamp'].dt.hour

Cuenta cuántas veces se cometen actos de imprudencia a cada hora

In [ ]:
conteo_por_hora = vehiculos_sobrepasan_limite['hora'].value_counts()

Ordena las horas en orden ascendente

In [ ]:
conteo_por_hora = conteo_por_hora.sort_index()

Crea el gráfico de barras

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(conteo_por_hora.index, conteo_por_hora.values, color='skyblue')
plt.xlabel('Hora del Día')
plt.ylabel('Número de Excesos de Velocidad')
plt.title('Número de Excesos de Velocidad a lo Largo del Día')
plt.xticks(range(24))  # Mostrar todas las horas del día en el eje X
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
# %%
# Cuál es la ruta que más veces sobrepasa el límite de velocidad
#
ruta_1_df['id_unidad'].value_counts().sort_values()
#%%
# RESULTADOS GENERALES
# Cuál es el tipo de vehiculo que aparece más en estos vehiculos problematicos
# A que hora es en la que se cometen estos actos de imprudencia
# Filtra el DataFrame para obtener solo las filas donde la velocidad supera el límite
velocidad_limite = 48  # Supongamos que el límite de velocidad es de 48 km/h
vehiculos_sobrepasan_limite = df_new[df_new['velocidad'] > velocidad_limite]

Luego, cuentas cuántas veces aparece cada ruta en este subconjunto

In [ ]:
ruta_mas_sobrepasada = vehiculos_sobrepasan_limite['ruta'].value_counts().idxmax()
num_veces = vehiculos_sobrepasan_limite['ruta'].value_counts().max()
print("La ruta que más veces sobrepasa el límite de velocidad en df_new es:", ruta_mas_sobrepasada, "y lo sobrepasa", num_veces)

%%

In [ ]:
vehiculos_sobrepasan_limite = df_new[df_new['velocidad'] > velocidad_limite]

Luego, cuentas cuántas veces aparece cada ruta en este subconjunto

In [ ]:
top_10_rutas = vehiculos_sobrepasan_limite['ruta'].value_counts().nlargest(10)

Crea un gráfico de barras para mostrar las 10 rutas con más ocurrencias

In [ ]:
plt.figure(figsize=(12, 6))
top_10_rutas.plot(kind='bar', color='skyblue')
plt.title('Top 10 Rutas con Mayor Exceso del Límite de Velocidad')
plt.xlabel('Ruta')
plt.ylabel('Número de Ocurrencias')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plt.show()

%%